In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


# WebScraping

In [5]:
url = "https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29"

response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

table = soup.find("table", {"class": "wikitable"})

In [7]:
table_head = ["Country", "Region", "IMF Estimate", "Year"]

In [ ]:
table_rows = table.find("tbody").find_all("tr")[3:]
df = {
    "Country": [],
    "Region": [],
    "IMF Estimate": [],
    "Year": []
}

for row in table_rows:
    row_data = row.find_all("td")
    temp = []
    # check if row value is empty
    df["Country"].append(row_data[0].text)
    df["Region"].append(row_data[1].text)

    if row_data[2].text == "—":
        print()
        df["IMF Estimate"].append(None)
        df["Year"].append(None)
    else:
        df["IMF Estimate"].append(int(row_data[2].text.replace(",", "")))
        df["Year"].append(int(row_data[3].text[-4:]))





In [36]:
for key in df:
    print(len(df[key]))

213
213
213
213


# Transformation

In [38]:
df = pd.DataFrame(df)

In [40]:
def millions_to_billions(value):
    """
    Convert a value from millions to billions and round to two decimal places.
    """
    return round(value / 1000, 2)

In [41]:
df.rename(columns={"IMF Estimate": "GDP (billion USDs)"}, inplace=True)

df["GDP (billion USDs)"] = df["GDP (billion USDs)"].apply(millions_to_billions)

In [49]:
df["Year"] = df["Year"].astype("Int32")

Country                0
Region                 0
GDP (billion USDs)    22
Year                  22
dtype: int64

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json
import sqlite3
import logging

# Setup logging
logging.basicConfig(filename='etl_project_log.txt', level=logging.INFO, 
                    format='%(asctime)s - %(levelname)s - %(message)s')

def scrape_gdp_data():
    url = "https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    table = soup.find("table", {"class": "wikitable"})
    table_rows = table.find("tbody").find_all("tr")[3:]
    
    df = {
        "Country": [],
        "Region": [],
        "IMF Estimate": [],
        "Year": []
    }

    for row in table_rows:
        row_data = row.find_all("td")
        df["Country"].append(row_data[0].text.strip())
        df["Region"].append(row_data[1].text.strip())
        if row_data[2].text.strip() == "—":
            df["IMF Estimate"].append(None)
            df["Year"].append(None)
        else:
            df["IMF Estimate"].append(int(row_data[2].text.strip().replace(",", "")))
            df["Year"].append(int(row_data[3].text.strip()[-4:]))

    return df

def transform_data(df):
    df = pd.DataFrame(df)
    df.rename(columns={"IMF Estimate": "GDP (billion USDs)"}, inplace=True)
    
    def millions_to_billions(value):
        return round(value / 1000, 2) if value else None

    df["GDP (billion USDs)"] = df["GDP (billion USDs)"].apply(millions_to_billions)
    df["Year"] = df["Year"].astype("Int32")
    return df

def save_to_csv(df, filename):
    df.to_csv(filename, index=False)

def save_to_db(df, db_name, table_name):
    conn = sqlite3.connect(db_name)
    df.to_sql(table_name, conn, if_exists='replace', index=False)
    conn.close()

def query_db(db_name, query):
    conn = sqlite3.connect(db_name)
    result = pd.read_sql_query(query, conn)
    conn.close()
    return result

def main():
    logging.info("Starting the ETL process")

    try:
        logging.info("Scraping data")
        raw_data = scrape_gdp_data()
        logging.info("Scraping data completed")

        logging.info("Transforming data")
        transformed_data = transform_data(raw_data)
        logging.info("Data transformation completed")

        csv_filename = 'Countries_by_GDP.json'
        logging.info(f"Saving data to JSON file: {json_filename}")
        save_to_csv(transformed_data, csv_filename)
        logging.info(f"Data saved to JSON file: {json_filename}")

        db_name = 'World_Economies.db'
        table_name = 'Countries_by_GDP'
        logging.info(f"Saving data to database: {db_name}, table: {table_name}")
        save_to_db(transformed_data, db_name, table_name)
        logging.info(f"Data saved to database: {db_name}, table: {table_name}")

        query = "SELECT * FROM Countries_by_GDP WHERE `GDP (billion USDs)` > 100"
        logging.info("Querying database for economies with more than 100 billion USD GDP")
        result = query_db(db_name, query)
        logging.info("Query executed successfully")
        logging.info(f"Query result:\n{result}")

    except Exception as e:
        logging.error(f"Error occurred: {e}")
        raise

    logging.info("ETL process completed")

if __name__ == "__main__":
    main()


In [3]:
currency_exchange_rates = requests.get("https://open.er-api.com/v6/latest/USD").json()["rates"]


In [23]:
# extract table

data = requests.get("https://www.forbesindia.com/article/explainers/the-10-largest-banks-in-the-world/86967/1").text
soup = BeautifulSoup(data, "html.parser")

table = soup.find("table", {"style": "border-collapse: collapse;"})

In [24]:
table_rows = table.find("tbody").find_all("tr")

table_head = [i.text for i in table.find_all("th")]

In [56]:
df = {key: [] for key in table_head}

for row in table_rows[1:]:
    row_data = row.find_all("td")
    try:
        for i, key in enumerate(table_head, start=0):
            df[key].append(row_data[i].text)
    except Exception as e:
        print(e)
        continue

list index out of range


In [57]:
for key in df:
    print(len(df[key]))

10
10
10


In [58]:
df = pd.DataFrame(df)

In [59]:
df

,Rank & Bank,Headquarters,Market Cap
0,#1 JPMorgan Chase,"New York, USA",$551.03 B
1,#2 Bank of America,"North Carolina, USA",$288.96 B
2,#3 Industrial and Commercial Bank of China Lim...,"Beijing, China",$249.28 B
3,#4 Wells Fargo,"California, USA",$208.41 B
4,#5 Agricultural Bank of China,"Beijing, China",$207.79 B
5,#6 Bank of China,"Beijing, China",$171.35 B
6,#7 China Construction Bank,"Beijing, China",$166.19 B
7,#8 HSBC,"London, UK",$164.48 B
8,#9 Morgan Stanley,"New York, USA",$148.90 B
9,#10 HDFC Bank,"Mumbai, India",$147.31 B


In [60]:
df.rename(columns={"Market Cap": "Market Cap (billion USDs)"}, inplace=True)

In [61]:
df["Rank & Bank"] = df["Rank & Bank"].apply(lambda x: " ".join(x.split(" ")[1:]))

In [63]:
df["Market Cap (billion USDs)"] = df["Market Cap (billion USDs)"].apply(lambda x: float(x.replace("B", "").replace("$", "")))
df["Market Cap (billion USDs)"] = df["Market Cap (billion USDs)"].apply(lambda x: round(x, 2))

In [67]:
# exchnage rates for GBP, EUR, and INR
exchanges = ["GBP", "EUR", "INR"]

for i in exchanges:
    col_name = f"Market Cap (billion {i}s)"
    df[col_name] = df["Market Cap (billion USDs)"].apply(lambda x: round(x * currency_exchange_rates[i], 2))

In [68]:
df

,Rank & Bank,Headquarters,Market Cap (billion USDs),Market Cap (billion GBPs),Market Cap (billion EURs),Market Cap (billion INRs)
0,JPMorgan Chase,"New York, USA",551.03,432.71,508.04,45794.79
1,Bank of America,"North Carolina, USA",288.96,226.91,266.42,24014.78
2,Industrial and Commercial Bank of China Limited,"Beijing, China",249.28,195.75,229.83,20717.07
3,Wells Fargo,"California, USA",208.41,163.66,192.15,17320.46
4,Agricultural Bank of China,"Beijing, China",207.79,163.17,191.58,17268.93
5,Bank of China,"Beijing, China",171.35,134.56,157.98,14240.49
6,China Construction Bank,"Beijing, China",166.19,130.51,153.23,13811.66
7,HSBC,"London, UK",164.48,129.16,151.65,13669.54
8,Morgan Stanley,"New York, USA",148.90,116.93,137.28,12374.72
9,HDFC Bank,"Mumbai, India",147.31,115.68,135.82,12242.58
